In [1]:
import pandas as pd
import numpy as np
import os
import utils
from datetime import datetime
import importlib

In [2]:
df = pd.concat([pd.read_csv('hka-aqm-am/' + f.removeprefix('._'), skiprows=1, sep=';', engine='python') for f in os.listdir('hka-aqm-am/')])
df.shape


(608036, 18)

In [4]:
importlib.reload(utils)
performance_df = pd.read_csv('model_performances.csv')
d_model=256
nhead=8
num_layers=5 
dropout_pe=0.15
dropout_encoder=0.15
batch_size=128
learning_rate=0.00031
epochs=25
aggregation_level = 'quarter_hour'
y_feature = 'CO2'
    
model, scaler, rmse, mae, me, mape, train_loss, val_loss, n_features = utils.create_multivariate_transformer_model_for_feature(df, d_model=d_model, nhead=nhead, num_layers=num_layers, dropout_pe=dropout_pe, dropout_encoder=dropout_encoder, batch_size=batch_size, learning_rate=learning_rate, epochs=epochs, y_feature=y_feature, aggregation_level=aggregation_level)
performance_df = performance_df.append({'model_name': 'multivariate_transformer','aggregation_level': aggregation_level, 'y_feature': y_feature, 'n_features': n_features, 'rmse': rmse, 'mae': mae, 'me': me, 'mape': mape, 'd_model': d_model, 'nhead': nhead, 'num_layers': num_layers, 'dropout_pe': dropout_pe,'dropout_encoder': dropout_encoder, 'batch_size': batch_size, 'learning_rate': learning_rate, 'epochs': epochs, 'train_loss': train_loss, 'val_loss': val_loss, 'note': '2 decoder Linear layers with LeakyReLu, shuffled training data'}, ignore_index=True)
performance_df.to_csv('model_performances.csv', index=False)
        

/Users/timehmann/Library/Mobile Documents/com~apple~CloudDocs/Studium/Data_Science_Karlsruhe/Data_Science_Semester6/Internet_Of_Things/IoT_Project/utils.py:1051: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  full_preprocessed_df_unscaled = deepcopy(df_cpy)


training data cutoff:  2023-07-15 02:45:00
device_id              int8
tmp                 float64
hum                 float64
CO2                 float64
VOC                 float64
vis                 float64
IR                  float64
WIFI                float64
BLE                 float64
rssi                float64
channel_rssi        float64
channel_index       float64
spreading_factor    float64
bandwidth           float64
f_cnt               float64
isHoliday           float64
isExamTime          float64
weekday_sin         float64
weekday_cos         float64
month_sin           float64
month_cos           float64
time_sin            float64
time_cos            float64
semester_SS23       float64
semester_WS22/23    float64
semester_WS23/24    float64
group                 int64
dtype: object


/Users/timehmann/Library/Mobile Documents/com~apple~CloudDocs/Studium/Data_Science_Karlsruhe/Data_Science_Semester6/Internet_Of_Things/IoT_Project/utils.py:1091: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/timehmann/Library/Mobile Documents/com~apple~CloudDocs/Studium/Data_Science_Karlsruhe/Data_Science_Semester6/Internet_Of_Things/IoT_Project/utils.py:1092: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # drop unconvertible columns
/Users/timehmann/Library/Mobile Documents/com~apple~CloudDocs/Stud

Training data shape: torch.Size([244176, 20, 25]) torch.Size([244176]) torch.Size([244176, 1])
Testing data shape: torch.Size([62745, 20, 25]) torch.Size([62745]) torch.Size([62745, 1])
Shuffled Training data shape: torch.Size([245536, 20, 25]) torch.Size([245536]) torch.Size([245536, 1])
Shuffled Testing data shape: torch.Size([61385, 20, 25]) torch.Size([61385]) torch.Size([61385, 1])
26
same columns
same dtypes
Dataframe is already saved.


In [ ]:
import torch

device_ids = torch.tensor([0])

device = utils.get_device()

model = utils.load_transformer_model(y_feature="CO2", model_name = 'transformer_multivariate_quarter_hour_26f', device=device)
scaler = utils.load_scaler(y_feature="CO2", model_name = 'transformer_multivariate_quarter_hour_26f')

real_data = utils.load_dataframe(model_name='transformer_multivariate_quarter_hour_26f')
pd.set_option('display.max_columns', None)
real_data.iloc[7:27]

In [ ]:
data_df = real_data.iloc[7:27].drop(columns=['device_id', 'date_time_rounded'])

data_df_scaled = scaler.transform(data_df)

input_data = torch.tensor(data_df_scaled, dtype=torch.float32).view(-1, 20, data_df_scaled.shape[1])
print(input_data.shape)
device_ids = torch.tensor([0])

model.eval()
with torch.no_grad():
    output = model(input_data.to(device), device_ids.to(device))
    print(output)

    prediction = output.cpu().numpy().reshape(-1, 1)
    print(prediction)
    print(prediction.shape)
    zeroes_for_scaler = np.zeros((prediction.shape[0], 25))

    zeroes_for_scaler[:, 2] = prediction  # Insert predicted values into the correct column
    print(zeroes_for_scaler)
    inverse_transformed = scaler.inverse_transform(zeroes_for_scaler)
    predicted_unscaled = inverse_transformed[:, 2].round(2)
    print(predicted_unscaled)
    